In [19]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
#from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
#from sklearn.model_selection import GridSearchCV
#from sklearn.kernel_ridge import KernelRidge
# 주석 처리는 non-linear regression model인 svr에 관련된 import

In [20]:
store = pd.read_csv("data/commercial_with_xy.csv",encoding = "UTF-8")
store.head()

,기준_년_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,총_생활인구_수,집객시설_수,아파트_평균_면적,아파트_평균_시가,상권_변화_지표,...,운영_영업_개월_평균,폐업_영업_개월_평균,서울_운영_영업_개월_평균,서울_폐업_영업_개월_평균,구분,점포_수,매출_금액,매출_건수,엑스좌표_값,와이좌표_값
0,2017,A,골목상권,2110001,이북5도청사,139405.75,22.5,124.0,376996219.5,LH,...,83.75,60.25,98.75,51.25,개인_서비스업,116,3.230693e+07,398,196193,456641
1,2017,A,골목상권,2110001,이북5도청사,139405.75,22.5,124.0,376996219.5,LH,...,83.75,60.25,98.75,51.25,교육_및_사업자원_서비스업,128,2.660860e+07,166,196193,456641
2,2017,A,골목상권,2110001,이북5도청사,139405.75,22.5,124.0,376996219.5,LH,...,83.75,60.25,98.75,51.25,기타_도소매업,98,2.564806e+06,96,196193,456641
3,2017,A,골목상권,2110001,이북5도청사,139405.75,22.5,124.0,376996219.5,LH,...,83.75,60.25,98.75,51.25,생활용품_도소매업,151,3.070350e+07,358,196193,456641
4,2017,A,골목상권,2110001,이북5도청사,139405.75,22.5,124.0,376996219.5,LH,...,83.75,60.25,98.75,51.25,수리업,24,1.999051e+08,874,196193,456641


In [21]:
store.columns

Index(['기준_년_코드', '상권_구분_코드', '상권_구분_코드_명', '상권_코드', '상권_코드_명', '총_생활인구_수',
       '집객시설_수', '아파트_평균_면적', '아파트_평균_시가', '상권_변화_지표', '상권_변화_지표_명',
       '운영_영업_개월_평균', '폐업_영업_개월_평균', '서울_운영_영업_개월_평균', '서울_폐업_영업_개월_평균', '구분',
       '점포_수', '매출_금액', '매출_건수', '엑스좌표_값', '와이좌표_값'],
      dtype='object')

In [22]:
store = store[['상권_코드_명','총_생활인구_수','집객시설_수','점포_수','아파트_평균_면적','아파트_평균_시가','서울_운영_영업_개월_평균','서울_폐업_영업_개월_평균','매출_건수', "매출_금액",'구분']]

In [23]:
store = store.drop_duplicates()

연도별값이 중요하지 않아서 연도 column 제외 + 그로인해 생기는 duplicate되는 값 제거

In [24]:
store

,상권_코드_명,총_생활인구_수,집객시설_수,점포_수,아파트_평균_면적,아파트_평균_시가,서울_운영_영업_개월_평균,서울_폐업_영업_개월_평균,매출_건수,매출_금액,구분
0,이북5도청사,139405.75,22.5,116,124.0,376996219.5,98.75,51.25,398,3.230693e+07,개인_서비스업
1,이북5도청사,139405.75,22.5,128,124.0,376996219.5,98.75,51.25,166,2.660860e+07,교육_및_사업자원_서비스업
2,이북5도청사,139405.75,22.5,98,124.0,376996219.5,98.75,51.25,96,2.564806e+06,기타_도소매업
3,이북5도청사,139405.75,22.5,151,124.0,376996219.5,98.75,51.25,358,3.070350e+07,생활용품_도소매업
4,이북5도청사,139405.75,22.5,24,124.0,376996219.5,98.75,51.25,874,1.999051e+08,수리업
...,...,...,...,...,...,...,...,...,...,...,...
105310,중부시장(신중부시장),927336.00,18.0,17,15.0,148636943.0,121.25,55.50,308,3.510748e+07,숙박업
105311,중부시장(신중부시장),927336.00,18.0,25,15.0,148636943.0,121.25,55.50,375,1.061767e+07,스포츠_및_오락_서비스업
105312,중부시장(신중부시장),927336.00,18.0,1663,15.0,148636943.0,121.25,55.50,147094,7.184864e+09,식료품_도소매업
105313,중부시장(신중부시장),927336.00,18.0,243,15.0,148636943.0,121.25,55.50,19070,5.655092e+08,음식점_및_주점업


In [25]:
store.rename(columns={'서울_운영_영업_개월_평균': '영업_개월_평균', '서울_폐업_영업_개월_평균':'폐업_개월_평균','구분':'업종_명'},inplace=True)

In [26]:
store = store.reset_index(drop=True)

In [27]:
store

,상권_코드_명,총_생활인구_수,집객시설_수,점포_수,아파트_평균_면적,아파트_평균_시가,영업_개월_평균,폐업_개월_평균,매출_건수,매출_금액,업종_명
0,이북5도청사,139405.75,22.5,116,124.0,376996219.5,98.75,51.25,398,3.230693e+07,개인_서비스업
1,이북5도청사,139405.75,22.5,128,124.0,376996219.5,98.75,51.25,166,2.660860e+07,교육_및_사업자원_서비스업
2,이북5도청사,139405.75,22.5,98,124.0,376996219.5,98.75,51.25,96,2.564806e+06,기타_도소매업
3,이북5도청사,139405.75,22.5,151,124.0,376996219.5,98.75,51.25,358,3.070350e+07,생활용품_도소매업
4,이북5도청사,139405.75,22.5,24,124.0,376996219.5,98.75,51.25,874,1.999051e+08,수리업
...,...,...,...,...,...,...,...,...,...,...,...
105193,중부시장(신중부시장),927336.00,18.0,17,15.0,148636943.0,121.25,55.50,308,3.510748e+07,숙박업
105194,중부시장(신중부시장),927336.00,18.0,25,15.0,148636943.0,121.25,55.50,375,1.061767e+07,스포츠_및_오락_서비스업
105195,중부시장(신중부시장),927336.00,18.0,1663,15.0,148636943.0,121.25,55.50,147094,7.184864e+09,식료품_도소매업
105196,중부시장(신중부시장),927336.00,18.0,243,15.0,148636943.0,121.25,55.50,19070,5.655092e+08,음식점_및_주점업


In [28]:
from sklearn.linear_model import LinearRegression

def grouped_regression(data, y, x):

    # output, input variables
    Y = data[y]
    X = data[x]

    # Create linear regression object
    linreg = LinearRegression()

    # Fit the linear regression model
    model = linreg.fit(X, Y)

    # Get the intercept and coefficients
    intercept = model.intercept_
    coef = model.coef_
    result = [intercept, coef]
    
    return result

In [29]:
# GroupBy

grouped = store.groupby("업종_명")

# Apply the UDF of linear regression model by Group
lin_reg_coef = grouped.apply(grouped_regression, "매출_금액", ["총_생활인구_수", "집객시설_수", "점포_수", "아파트_평균_면적", "아파트_평균_시가", "영업_개월_평균", "폐업_개월_평균", "매출_건수"])

In [30]:
lin_reg_coef

업종_명
가전제품_및_통신기기_도소매업    [-101937169993.95515, [-71.5719973115806, -137...
개인_서비스업             [-483595599.7006744, [-22.741973670684636, 115...
교육_및_사업자원_서비스업      [-4082395236.066702, [-10.78234457807494, 2989...
기타_도소매업             [-3311760875.2699127, [-38.991157689558754, 14...
기타_서비스업             [-96358115.03589582, [-0.69817038936355, 48151...
부동산_및_임대업           [-3082397123.6638365, [-30.285082740746606, 24...
생활용품_도소매업           [-1164548385.1718218, [-85.46455588838336, 957...
수리업                 [-1883312239.8197422, [0.04217015831718456, 45...
숙박업                 [1533302355.284123, [-0.10248245154709082, 567...
스포츠_및_오락_서비스업       [1893570208.4828565, [-3.625278421813342, 1235...
식료품_도소매업            [-50240413738.96919, [-21.271010632495297, -28...
음식점_및_주점업           [1630857936.0899467, [-14.080622781125989, 223...
제조업                 [-20710626101.522617, [-82.32451307823592, -21...
중고상품_도소매업           [-4917833693.438675, [180.6392045539107, 10015...
dtype: object